# Группировка данных и оконные функции

## Подготовка

Таблицы transactions и customers были созданы ранее, поэтому только настроим коннекторы

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

# Креды для подключения к PostgreSQL. Пользователя и пароль берём из .env
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = "localhost"
port = 5432
database = "db1"

# Создаём движок для работы с базой
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Функция для создания схем и таблиц
def execute_query(sql):
    with engine.connect() as conection:
        try:
            conection.execute(text(sql))
            conection.commit()
            print("Запрос успешно выполнен! 🎉🎉🎉")
        except Exception as e:
            print(f"При выполнении запроса возникла ошибка: {e}")

## Задание 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества

In [2]:
# Выводим все строки
pd.set_option('display.max_rows', 80)

sql = """
select
	job_industry_category,
	count(distinct customer_id) as users
from store_2.customers
group by 1
order by 2 desc
"""

pd.read_sql(sql, con=engine)

,job_industry_category,users
0,Manufacturing,799
1,Financial Services,774
2,None,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


## Задание 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности

In [3]:
sql = """
select
	date_trunc('month', transaction_date) :: date as transaction_month,
	job_industry_category,
	count(distinct transaction_id) as transactions
from store_2.transactions as t
join store_2.customers as c on t.customer_id = c.customer_id
group by 1, 2
order by 1, 2
"""

pd.read_sql(sql, con=engine)

,transaction_month,job_industry_category,transactions
0,2017-01-01,Argiculture,41
1,2017-01-01,Entertainment,58
2,2017-01-01,Financial Services,319
3,2017-01-01,Health,261
4,2017-01-01,IT,100
...,...,...,...
115,2017-12-01,Manufacturing,299
116,2017-12-01,Property,117
117,2017-12-01,Retail,140
118,2017-12-01,Telecommunications,26


## Задание 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT

In [4]:
sql = """
select
	brand,
	count(distinct transaction_id) as transactions
from store_2.transactions as t
join store_2.customers as c on t.customer_id = c.customer_id
where 1 = 1
and online_order
and order_status = 'Approved'
and job_industry_category = 'IT'
group by 1
"""

pd.read_sql(sql, con=engine)

,brand,transactions
0,Giant Bicycles,89
1,Norco Bicycles,92
2,OHM Cycles,78
3,Solex,101
4,Trek Bicycles,82
5,WeareA2B,90
6,None,8


## Задание 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат

In [5]:
# Чистый group by
sql = """
select
	customer_id,
	sum(list_price) as total_sum,
	min(list_price) min_sum,
	max(list_price) max_sum,
	count(transaction_id) as transactions
from store_2.transactions
group by customer_id
order by total_sum desc, transactions desc
"""

group_by = pd.read_sql(sql, con=engine)
group_by

,customer_id,total_sum,min_sum,max_sum,transactions
0,2183,19071.32,230.91,2005.66,14
1,1129,18349.27,290.62,1992.93,13
2,1597,18052.68,360.40,2091.47,12
3,941,17898.46,1057.51,2091.47,10
4,2788,17258.94,183.86,2083.94,11
...,...,...,...,...,...
3489,2423,202.62,202.62,202.62,1
3490,3189,200.70,100.35,100.35,2
3491,2274,142.98,71.49,71.49,2
3492,2532,71.49,71.49,71.49,1


In [6]:
# Оконные функции
sql = """
select distinct
	customer_id,
	sum(list_price) over(partition by customer_id) as total_sum,
	min(list_price) over(partition by customer_id) min_sum,
	max(list_price) over(partition by customer_id) max_sum,
	count(transaction_id) over(partition by customer_id) as transactions
from store_2.transactions
order by total_sum desc, transactions desc
"""

window = pd.read_sql(sql, con=engine)
window

,customer_id,total_sum,min_sum,max_sum,transactions
0,2183,19071.32,230.91,2005.66,14
1,1129,18349.27,290.62,1992.93,13
2,1597,18052.68,360.40,2091.47,12
3,941,17898.46,1057.51,2091.47,10
4,2788,17258.94,183.86,2083.94,11
...,...,...,...,...,...
3489,2423,202.62,202.62,202.62,1
3490,3189,200.70,100.35,100.35,2
3491,2274,142.98,71.49,71.49,2
3492,2532,71.49,71.49,71.49,1


In [7]:
# Сравниваем результаты. Сначала зададим customer_id в качестве индекса, чтобы сверять по клиентам
group_by = group_by.set_index("customer_id").sort_index()
window = window.set_index("customer_id").sort_index()
# Выводим только те строки, которые отличаются. round добавляем из - за неточностей округления при агрегации
group_by.round(2).compare(window.round(2), keep_equal=False)

customer_id


Как видим, вывод через group by и через оконные функции идентичен (при использовании distinct)

## Задание 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы

In [8]:
# Запрос для максимальной суммы
# Для каждого пользователя находим сумму транзакция. Оставляем только те customer_id, у которых сумма транзакций такая же, как максимальная среди всех
sql = """
with
max_sum as (
	select
		customer_id,
		max(transaction_sum) over() as max_sum_value,
		transaction_sum = max(transaction_sum) over() as was_max_sum
	from (
		select
			customer_id,
			sum(list_price) as transaction_sum
		from store_2.transactions
		group by 1
	)
)

select
	first_name,
	last_name,
	max_sum_value
from store_2.customers
join max_sum using (customer_id)
where was_max_sum
"""

pd.read_sql(sql, con=engine)

,first_name,last_name,max_sum_value
0,Jillie,Fyndon,19071.32


In [9]:
# Запрос для минимальной суммы
# Для каждого пользователя находим сумму транзакция. Оставляем только те customer_id, у которых сумма транзакций такая же, как минимальная среди всех
sql = """
with
min_sum as (
	select
		customer_id,
		min(transaction_sum) over() as min_sum_value,
		transaction_sum = min(transaction_sum) over() as was_min_sum
	from (
		select
			customer_id,
			sum(list_price) as transaction_sum
		from store_2.transactions
		group by 1
        having sum(list_price) IS NOT NULL
	)
)

select
	first_name,
	last_name,
	min_sum_value
from store_2.customers
join min_sum using (customer_id)
where was_min_sum
"""

pd.read_sql(sql, con=engine)

,first_name,last_name,min_sum_value
0,Hamlen,Slograve,60.34


## Задание 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций

In [10]:
sql = """
select
--	distinct on (customer_id) -- раскомментить, если нужно не более одной транзакции на пользователя
	*
from (
	select
		*,
		rank() over(partition by customer_id order by transaction_date) as r, -- вариан
		min(transaction_date) over(partition by customer_id) as first_tr_date
	from store_2.transactions
)
where 1 = 1
-- AND r = 1
AND transaction_date = first_tr_date
order by customer_id, list_price DESC
"""

pd.read_sql(sql, con=engine)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,r,first_tr_date
0,9785,72,1,2017-01-05,False,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,1,2017-01-05
1,2261,1,2,2017-05-04,True,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,1,2017-05-04
2,10302,33,3,2017-02-23,False,Approved,Giant Bicycles,Standard,medium,small,1311.44,1167.18,1,2017-02-23
3,12441,95,4,2017-04-03,False,Approved,Giant Bicycles,Standard,medium,large,569.56,528.43,1,2017-04-03
4,2291,23,5,2017-03-03,True,Approved,Norco Bicycles,Mountain,low,small,688.63,612.88,1,2017-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,8276,18,3497,2017-09-01,True,Approved,Solex,Standard,medium,medium,575.27,431.45,1,2017-09-01
3516,13469,12,3498,2017-02-03,True,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,1,2017-02-03
3517,2794,62,3499,2017-01-12,False,Approved,Solex,Standard,medium,medium,478.16,298.72,1,2017-01-12
3518,6309,69,3500,2017-01-09,True,Approved,Giant Bicycles,Road,medium,medium,792.90,594.68,1,2017-01-09


## Задание 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [11]:
"""
Условие задания не совсем понятно, поэтому укажу какую задачу решает запрос:
1) Для каждого customer_id находим интервал между текущей и предыдущей транзакцией (колонка dt_interval)
2) Из всех интервалов находим максимальный (колонка max_dt_interval)
3) Оставляем только те customer_id, у которых интервал равен максимальному интервалу среди клиентов (dt_interval = max_dt_interval)
4) По этим customer_id выводим имя, фамилию, профессию
"""

sql = """
select
	first_name,
	last_name,
	job_title
from store_2.customers
where customer_id in (
	select distinct
		customer_id
	from (
		select
			customer_id,
			dt_interval,
			max(dt_interval) over() as max_dt_interval
		from (
			select
				customer_id,
				transaction_date - lag(transaction_date) over(partition by customer_id order by transaction_date) as dt_interval
			from store_2.transactions
		)
	)
	where dt_interval = max_dt_interval
)
"""

pd.read_sql(sql, con=engine)

,first_name,last_name,job_title
0,Susanetta,None,Legal Assistant
